In [47]:
import os, sys
from pyspark.sql import SparkSession
import pyspark.sql.functions as fun
from pyspark.sql.functions import col
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [48]:
os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

In [49]:
spark = SparkSession.builder.master("local[*]").appName("ML_LinearRegression").getOrCreate()

In [53]:
path=r"C:\Users\ACER\OneDrive\Desktop\spark_lab\data\data.csv"

In [60]:
df = spark.read.csv(path, sep="\t", header=True)


In [54]:
#df = spark.read.csv(path, header=True)

In [61]:
df.show(5)

+----+----+---+---+---+-----+---+---------+----+-----------+----------+-------------------+------------+------+
|Runs|Mins| BF| 4s| 6s|   SR|Pos|Dismissal|Inns| Opposition|    Ground|Start DateAscending|Match Number|Result|
+----+----+---+---+---+-----+---+---------+----+-----------+----------+-------------------+------------+------+
|  11|  92|  -|  -|  0|    -|  6|      lbw|   1|v Sri Lanka|Chandigarh|          23-Nov-90| Test # 1156|   Won|
|  50| 158|118|  6|  0|42.37|  4|   caught|   1|  v England|   Kolkata|          29-Jan-93| Test # 1211|   Won|
|   9|  17| 19|  2|  0|47.36|  4|  not out|   4|  v England|   Kolkata|          29-Jan-93| Test # 1211|   Won|
| 165| 361|296| 24|  1|55.74|  4|   caught|   1|  v England|   Chennai|          11-Feb-93| Test # 1213|   Won|
|  78| 285|213| 10|  0|36.61|  4|      lbw|   2|  v England|    Mumbai|          19-Feb-93| Test # 1214|   Won|
+----+----+---+---+---+-----+---+---------+----+-----------+----------+-------------------+------------+

In [62]:
df = df.toDF(*[col.strip() for col in df.columns])


In [65]:
df.show(10)

+----+----+---+---+---+-----+---+---------+----+-----------+-------------+-------------------+------------+------+
|Runs|Mins| BF| 4s| 6s|   SR|Pos|Dismissal|Inns| Opposition|       Ground|Start DateAscending|Match Number|Result|
+----+----+---+---+---+-----+---+---------+----+-----------+-------------+-------------------+------------+------+
|  11|  92|  -|  -|  0|    -|  6|      lbw|   1|v Sri Lanka|   Chandigarh|          23-Nov-90| Test # 1156|   Won|
|  50| 158|118|  6|  0|42.37|  4|   caught|   1|  v England|      Kolkata|          29-Jan-93| Test # 1211|   Won|
|   9|  17| 19|  2|  0|47.36|  4|  not out|   4|  v England|      Kolkata|          29-Jan-93| Test # 1211|   Won|
| 165| 361|296| 24|  1|55.74|  4|   caught|   1|  v England|      Chennai|          11-Feb-93| Test # 1213|   Won|
|  78| 285|213| 10|  0|36.61|  4|      lbw|   2|  v England|       Mumbai|          19-Feb-93| Test # 1214|   Won|
|  62| 128|114|  7|  0|54.38|  4|   caught|   1| v Zimbabwe|        Delhi|      

In [77]:
df=df.dropna()

In [78]:
df.show(10)

+-----+-----+-----+---+---+-----+---+---------+----+-----------+-------------+-------------------+------------+------+
| Runs| Mins|   BF| 4s| 6s|   SR|Pos|Dismissal|Inns| Opposition|       Ground|Start DateAscending|Match Number|Result|
+-----+-----+-----+---+---+-----+---+---------+----+-----------+-------------+-------------------+------------+------+
| 50.0|158.0|118.0|  6|  0|42.37|  4|   caught|   1|  v England|      Kolkata|          29-Jan-93| Test # 1211|   Won|
|  9.0| 17.0| 19.0|  2|  0|47.36|  4|  not out|   4|  v England|      Kolkata|          29-Jan-93| Test # 1211|   Won|
|165.0|361.0|296.0| 24|  1|55.74|  4|   caught|   1|  v England|      Chennai|          11-Feb-93| Test # 1213|   Won|
| 78.0|285.0|213.0| 10|  0|36.61|  4|      lbw|   2|  v England|       Mumbai|          19-Feb-93| Test # 1214|   Won|
| 62.0|128.0|114.0|  7|  0|54.38|  4|   caught|   1| v Zimbabwe|        Delhi|          13-Mar-93| Test # 1218|   Won|
| 28.0| 74.0| 52.0|  5|  0|53.84|  4|   caught| 

In [79]:
df = df.withColumn("Runs", df["Runs"].cast("float")) \
       .withColumn("Mins", df["Mins"].cast("float")) \
       .withColumn("BF", df["BF"].cast("float"))

In [80]:
df.show(10)

+-----+-----+-----+---+---+-----+---+---------+----+-----------+-------------+-------------------+------------+------+
| Runs| Mins|   BF| 4s| 6s|   SR|Pos|Dismissal|Inns| Opposition|       Ground|Start DateAscending|Match Number|Result|
+-----+-----+-----+---+---+-----+---+---------+----+-----------+-------------+-------------------+------------+------+
| 50.0|158.0|118.0|  6|  0|42.37|  4|   caught|   1|  v England|      Kolkata|          29-Jan-93| Test # 1211|   Won|
|  9.0| 17.0| 19.0|  2|  0|47.36|  4|  not out|   4|  v England|      Kolkata|          29-Jan-93| Test # 1211|   Won|
|165.0|361.0|296.0| 24|  1|55.74|  4|   caught|   1|  v England|      Chennai|          11-Feb-93| Test # 1213|   Won|
| 78.0|285.0|213.0| 10|  0|36.61|  4|      lbw|   2|  v England|       Mumbai|          19-Feb-93| Test # 1214|   Won|
| 62.0|128.0|114.0|  7|  0|54.38|  4|   caught|   1| v Zimbabwe|        Delhi|          13-Mar-93| Test # 1218|   Won|
| 28.0| 74.0| 52.0|  5|  0|53.84|  4|   caught| 

In [82]:
df=df.select(*["Runs","Mins","BF"])

In [83]:
X = df.drop("Runs")

In [85]:
from pyspark.sql.functions import col

In [86]:
assembler = VectorAssembler(inputCols=X.columns, outputCol="features")

In [87]:
df_vector = assembler.transform(df)

In [88]:
df_vector.show(5)

+-----+-----+-----+-------------+
| Runs| Mins|   BF|     features|
+-----+-----+-----+-------------+
| 50.0|158.0|118.0|[158.0,118.0]|
|  9.0| 17.0| 19.0|  [17.0,19.0]|
|165.0|361.0|296.0|[361.0,296.0]|
| 78.0|285.0|213.0|[285.0,213.0]|
| 62.0|128.0|114.0|[128.0,114.0]|
+-----+-----+-----+-------------+
only showing top 5 rows



In [90]:
output=df_vector.select("features","Runs")

In [91]:
output.show(2)

+-------------+----+
|     features|Runs|
+-------------+----+
|[158.0,118.0]|50.0|
|  [17.0,19.0]| 9.0|
+-------------+----+
only showing top 2 rows



In [92]:
regression = LinearRegression(featuresCol="features", labelCol="Runs")

In [93]:
model=regression.fit(output)

In [95]:
model.coefficients

DenseVector([0.1825, 0.302])

In [96]:
predictions=model.transform(df_vector)

In [97]:
predicted_df = predictions.select("Runs", "Mins", "BF", "prediction").toPandas()

In [100]:
out=predicted_df.to_csv()

In [101]:
print(predicted_df)

      Runs   Mins     BF  prediction
0     50.0  158.0  118.0   62.943103
1      9.0   17.0   19.0    7.312663
2    165.0  361.0  296.0  153.746875
3     78.0  285.0  213.0  114.810608
4     62.0  128.0  114.0   56.260288
..     ...    ...    ...         ...
323  146.0  465.0  314.0  178.162274
324   14.0  146.0   91.0   52.598921
325   94.0  219.0  153.0   84.645498
326    3.0    6.0    7.0    1.681125
327    2.0   25.0   13.0    6.960543

[328 rows x 4 columns]
